##LangChain Memory – Theoretical Overview

LangChain provides a variety of memory modules to help chains and agents **retain state or context across runs**. Each type is optimized for different scenarios and trade‑offs.

---

## 1. `SimpleMemory`
- **Purpose**: Store fixed context or static key-value data that **never changes** during sessions.
- **Use‑case**: Injecting static user profile, preferences, or configuration info into every prompt.
- **Main difference**: Immutable store; no temporal or conversational retention :contentReference[oaicite:0]{index=0}.
- **Note**: Not for conversation like memory for remembering previous question.

In [1]:
# ====================================
# Step 1: Install Dependencies
# ====================================
!pip install -q langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00


In [2]:

# ====================================
# Step 2: Imports
# ====================================
from langchain_groq import ChatGroq
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import SimpleMemory
from langchain_core.runnables import RunnableWithMessageHistory
from IPython.display import Markdown, display

In [14]:
# ====================================
# Step 3: Setup Groq Chat LLM
# ====================================
from google.colab import userdata
api_key = userdata.get("GROQ_API_KEY")  # or hardcode your API key
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=api_key)

In [15]:
# ====================================
# Step 4: Create a Prompt Template
# ====================================
system_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. The user's name is {user_name} and they are a {user_role}. "
    "Always greet them by name and tailor responses based on their role."
)

# 🧠 This is where memory will inject static user context
human_msg = HumanMessagePromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_messages([
    system_msg,
    # MessagesPlaceholder(variable_name="history"),  # Placeholder for memory
    human_msg
])

In [16]:
# ====================================
# Step 5: Setup SimpleMemory
# ====================================
memory = SimpleMemory(memories={
    "user_name": "Dhruv",
    "user_role": "Data Scientist"
})

In [23]:
# ====================================
# Step 6: Chain LLM + Prompt + Memory
# ====================================
chain = prompt | llm

# Wrap with message history using memory
chat = RunnableWithMessageHistory(
    chain,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="history" # ChatMessageHistory handles this internally
)

In [25]:
# ====================================
#  Step 7: Ask Questions with SimpleMemory
# ====================================
user_inputs = [
    "What can you help me with today?",
    "Give me advice on improving model accuracy.",
]

for input_text in user_inputs:
    prompt_input = {
        "user_name": memory.memories["user_name"],
        "user_role": memory.memories["user_role"],
        "input": input_text
    }

    messages = prompt.format_messages(**prompt_input)
    response = llm.invoke(messages)

    display(Markdown(f"### ❓ User: {input_text}"))
    display(Markdown(f"**🤖 LLM Response:** {response.content}"))

### ❓ User: What can you help me with today?

**🤖 LLM Response:** Hello Dhruv, I'm glad you asked. As a Data Scientist, you likely work with complex data sets, models, and algorithms on a daily basis. I can assist you with a wide range of tasks, such as:

* Providing explanations and examples of various machine learning concepts and techniques
* Helping with data preprocessing, feature engineering, and data visualization
* Offering guidance on model selection, hyperparameter tuning, and model evaluation
* Discussing best practices for working with popular data science libraries and frameworks, such as Python, R, or SQL
* Brainstorming ideas for tackling specific data science problems or projects
* Even just being a sounding board for your ideas or helping you troubleshoot issues with your code

What's on your mind today, Dhruv? Do you have a specific project or problem you'd like some help with?

### ❓ User: Give me advice on improving model accuracy.

**🤖 LLM Response:** Hello Dhruv, I hope you're doing well. As a Data Scientist, you're likely no stranger to the challenges of improving model accuracy. Here are some tips that might help:

1. **Data Quality Check**: Ensure that your dataset is clean, complete, and consistent. Handling missing values, outliers, and data normalization can significantly impact model performance.
2. **Feature Engineering**: Identify the most relevant features that contribute to the model's predictions. You can use techniques like correlation analysis, mutual information, or recursive feature elimination to select the best features.
3. **Hyperparameter Tuning**: Perform grid search, random search, or Bayesian optimization to find the optimal hyperparameters for your model. This can be time-consuming, but it's often crucial for achieving better accuracy.
4. **Model Selection**: Experiment with different algorithms and models to find the one that best suits your problem. Consider ensemble methods, such as bagging, boosting, or stacking, which can often improve accuracy.
5. **Cross-Validation**: Use techniques like k-fold cross-validation to evaluate your model's performance on unseen data. This helps prevent overfitting and gives you a more realistic estimate of the model's accuracy.
6. **Regularization**: Apply regularization techniques, such as L1 or L2 regularization, to prevent overfitting and improve the model's generalizability.
7. **Ensemble Methods**: Combine the predictions of multiple models to improve overall accuracy. This can be particularly effective when dealing with complex datasets or noisy data.
8. **Data Augmentation**: If you're working with image or text data, consider using data augmentation techniques to increase the size and diversity of your dataset.
9. **Monitor Performance Metrics**: Track metrics like precision, recall, F1-score, and ROC-AUC to get a comprehensive understanding of your model's performance.
10. **Iterate and Refine**: Continuously refine your model by iterating through these steps, and be willing to try new approaches and techniques to improve accuracy.

Remember, Dhruv, improving model accuracy is often an iterative process that requires patience, persistence, and a deep understanding of the data and the problem you're trying to solve.

### 🧩 Summary Table

| Memory Type                   | Retention Style         | Use‑Case Scenario                              | Key Trade‑Offs                |
|------------------------------|--------------------------|------------------------------------------------|-------------------------------|
| `SimpleMemory`               | Static key-value         | System configs, user profile                  | No recall after changes       |
| `ConversationBufferMemory`   | Full chat transcript     | Small-scale chatbots                          | Token limits on long chats    |
| `ConversationSummaryMemory`  | Summarized history       | Extended multi-turn dialogues                 | Requires summarization LLM    |
| `VectorStoreRetrieverMemory` | Semantic retrieval       | Agents needing recollection from long history | Extra overhead, complexity    |
| `CombinedMemory`            | Composite memory         | Complex agents with varied memory needs       | Must manage & sync components |

---

## 🧠 Choosing the Right Memory

- **Short conversations or testing** → use `ConversationBufferMemory`
- **Extended sessions or long-term chatbots** → use `ConversationSummaryMemory`
- **Hybrid pipelines needing static data + context** → use `CombinedMemory`
- **Semantic retrieval over long history** → use `VectorStoreRetrieverMemory`
- **Fixed metadata/config injection** → use `SimpleMemory`
